In [1]:
import gc
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
from pyhive import presto

In [2]:
warnings.filterwarnings('ignore')

In [3]:
def get_query(date: str, service_area_id: int = 1, cct_id: int = 121) -> str:
    return f"""
        select
            booking_id,
            assignment_time,
            estimated_trip_distance as distance,
            estimated_duration as duration,
            peak_captain,
            peak_customer,
            max_estimated_price as price
        from prod_dwh.booking
        where 1=1
         and service_area_id = {service_area_id}
         and day = date('{date}')
         and cct_id = {cct_id}
         and is_trip_ended
         and not is_corporate
         and not is_hdl
         and not is_intercity
         and not is_later
         and estimated_trip_distance is not null
         and estimated_duration is not null
         and max_estimated_price is not null
         and max_estimated_price > 0.0
    """

In [4]:
conn = presto.connect(
    host='presto-python-r-script-cluster.careem-engineering.com',
    username='presto_python_r',
    port=8080
)

In [5]:
date_range = [str(x).split(' ')[0] for x in pd.date_range(start='2023-02-01', end='2023-04-20', freq='D')]

In [6]:
len(date_range)

79

In [7]:
df_list = []

for date in tqdm(date_range):
    sub = pd.read_sql(sql=get_query(date, 3, 171), con=conn)
    df_list.append(sub)
    
data = pd.concat(df_list, ignore_index=True)

del sub
del df_list
gc.collect()

100%|███████████████████████████████████████████| 79/79 [04:42<00:00,  3.57s/it]


0

In [8]:
data.head()

,booking_id,assignment_time,distance,duration,peak_captain,peak_customer,price
0,1696220166,2023-02-01 13:23:06.000,1.87405,5,1.4,1.4,15.07
1,1695907575,2023-02-01 06:08:06.000,7.46549,10,1.0,1.0,20.11
2,1695997244,2023-02-01 08:55:33.000,1.23862,3,1.0,1.0,10.93
3,1696458671,2023-02-01 19:01:42.000,6.37642,9,1.0,1.0,17.24
4,1696437981,2023-02-01 18:12:49.000,5.67880,12,1.0,1.0,8.69


In [9]:
len(data)

596552

In [10]:
data.to_parquet('data/riyadh.pq')